### Homework 3 - Random matrices ###

1) Choose a given type of ecological structure (e.g. mutualistic, or predator-prey, or etc...). Generate random matrices with for completely random and for the chosen ecological structures. You need to create SxS matrices (S is the number of species) with C non-zero entries and 1-C zeros (C is the connectivity between 0 and 1). The non-zero elements are drawn at random from given distributions. Depending on the network structure some symmetries and constraints may hold. Please follow the detailed step-by-step explanation in the "Homework-detail-week3-from-Allesina-Stability-Criteria-2012-Nature.pdf" (Also uploaded in the Google Drive, note folder). Fix C and variance of the interaction and S=100, and calculate the maximum real eigenvalue of the matrix. Do many realizations and calculate each time such maximum real eigenvalue. Do this for different combinations of C and of the  variance so to have a good range of values of the control parameter  (e.g. for the complete random case is sigma*(S*C)^0.5). Set the self interaction d=-1

Plot the probability for the maximum real eigenvalue of be smaller than 0, against the corresponding control parameter and see when the transition to instability happens. Compare with analytical prediction if avaliable.

In [1]:
import numpy as np
import matplotlib.pyplot as pltf

In [ ]:
# Let's choose predator-prey dynamics

## PARAMETERS
S = 100 # number of species
C = 0.1 # connectance
d = -1 # self interaction (diagonal terms)
sigma2 = 1 # variance of the Normal distribubtion

# Set the seed for reproducibility (12345)
np.random.seed(12345)

# Generate completely random matrices (no ecological structures)
def random_mat(S, C, d, sigma2):
    
    mat = np.zeros((S,S)) # create an SxS matrix of zeros
    
    for i in range(S):
        mat[i,i] = -d                                       # set diagonal terms to -d
        for j in range(S):
            if i != j:                                      # non-diagonal terms
                p = np.random.uniform(0,1,1)                # extract a probability p of nodes i,j to be connected
                if p <= C:                                  # compare it with the connectance requirment
                    mat[i,j] = np.random.normal(0,sigma2,1) # extract the coefficient value                             
    
    return mat
    
# Predator-prey case
def predator_prey(S, C, d, sigma2):
    
    mat = np.zeros((S,S)) # create an SxS matrix of zeros
    
    for i in range(1,S):
        mat[i,i] = -d                                                       # set diagonal terms to -d
        for j in range(S):
            if i > j:                                                   # non-diagonal terms
                p1 = np.random.uniform(0,1,1)                           # extract a probability p of nodes i,j to be connected
                if p1 <= C:
                    p2 = np.random.uniform(0,1,1)                       # extract second probability
                    if p2 <= 0.5: 
                        mat[i,j] = np.abs(np.random.normal(0,sigma2,1))     # extract the coefficient value
                        mat[j,i] = np.abs(np.random.normal(0,sigma2,1))
                    else:
                        mat[i,j] = - np.abs(np.random.normal(0,sigma2,1))   # extract the coefficient value
                        mat[j,i] = - np.abs(np.random.normal(0,sigma2,1))
    
    return mat

## TEST 
np.set_printoptions(precision=2, suppress=True)             # for readability
print('Random matrice: \n', random_mat(50, 0.5, d, sigma2))

print('Predator-prey: \n', predator_prey(50,0.5,d,sigma2))


Random matrice: 
 [[ 1.    0.   -0.52 ...  0.29  0.    0.  ]
 [ 0.    1.    0.   ...  0.22 -0.19  0.  ]
 [ 0.96  0.    1.   ...  1.76 -0.4  -0.29]
 ...
 [ 1.89  0.    0.89 ...  1.   -0.49 -0.82]
 [ 0.    0.    0.2  ...  0.    1.    1.28]
 [-1.01  0.   -0.95 ... -1.26  0.    1.  ]]
Predator-prey: 
 [[ 0.    0.   -1.64 ...  0.    0.89  0.12]
 [ 0.    1.    0.   ...  0.3  -1.16 -0.09]
 [-0.69  0.    1.   ...  0.    0.    0.  ]
 ...
 [ 0.    0.08  0.   ...  1.    0.   -1.6 ]
 [ 0.71 -1.22  0.   ...  0.    1.    0.  ]
 [ 0.8  -1.02  0.   ... -1.34  0.    1.  ]]
Predator-prey: 
 0.15415622130823475


/tmp/ipykernel_3862/461042503.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  mat[i,j] = np.random.normal(0,sigma2,1) # extract the coefficient value
/tmp/ipykernel_3862/461042503.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  mat[i,j] = - np.abs(np.random.normal(0,sigma2,1))   # extract the coefficient value
/tmp/ipykernel_3862/461042503.py:44: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  mat[j,i] = - np.abs(np.random.normal(0,sigma2,1))
/tmp/ipykernel